In [1]:
import itertools

In [2]:
import pandas as pd
import random
import itertools

from ptrail.core.Datasets import Datasets
from ptrail.core.TrajectoryDF import PTRAILDataFrame
from ptrail.features.kinematic_features import KinematicFeatures
from ptrail.preprocessing.statistics import Statistics
from src.augmentation.augment import Augmentation
from src.selection.select import Selection

import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score, accuracy_score, average_precision_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.metrics import f1_score, accuracy_score, average_precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [3]:
# Load the dataset and create the distance column in it which is necessary for Augmentation.
starkey_data = Datasets.load_starkey()
ready_dataset = KinematicFeatures.create_distance_column(starkey_data).reset_index()

------------------------ Dataset Facts ------------------------------

Number of unique Trajectories in the data: 253
Number of points in the data: 287136
Dataset time range: 1196 days 22:51:45
Datatype of the DataFrame: <class 'ptrail.core.TrajectoryDF.PTRAILDataFrame'>
Dataset Bounding Box: (45.18896978643169, -118.61020848239596, 45.314545642992, -118.50455596234036)

---------------------------------------------------------------------


In [4]:
traj_ids = list(ready_dataset['traj_id'].unique())
train_size = int(len(traj_ids) * 0.90)

train_traj_ids = random.sample(traj_ids, train_size)
test_traj_ids = []
for t in traj_ids:
    if t not in train_traj_ids:
        test_traj_ids.append(t)

training, testing = ready_dataset.loc[ready_dataset.traj_id.isin(train_traj_ids)], \
    ready_dataset.loc[ready_dataset.traj_id.isin(test_traj_ids)]

In [5]:
# # Random selection
selected = Selection.select_randomly(training, k=0.046)
selected

['OSUX91063',
 '940212D01',
 '930406D01',
 'OSUX93110',
 '950124E01',
 '921216E04',
 '921230E03',
 'OSUX89146',
 '960313E02',
 'OSUX89141']

In [6]:
# Proportional selection
proportionally_selected = Selection.select_trajectories_proportionally(training, classification_col='Species', k=0.2)
proportionally_selected

Class: 0, Total trajectories in class: 46, trajectories selected: 10.0
Class: 1, Total trajectories in class: 114, trajectories selected: 23.0
Class: 2, Total trajectories in class: 67, trajectories selected: 14.0
47


['940117D02',
 '940217D01',
 '940316D01',
 '940105D02',
 '890130D09',
 '960121D01',
 '930218D01',
 '920303D05',
 '950126D01',
 '930415D02',
 '910315E22',
 '910313E19',
 '950405E12',
 '940219E02',
 '890328E12',
 '930410E01',
 '910319E11',
 '921230E03',
 '890418E13',
 '910319E02',
 '900205E01',
 '930304E13',
 '921215E02',
 '931215E09',
 '910313E11',
 '930202E07',
 '930304E16',
 '940213E01',
 '930203E03',
 '921228E09',
 '930410E04',
 '941220E09',
 '921228E16',
 'OSUX89065',
 'OSUX89087',
 'OSUX91164',
 'OSUX93039',
 'OSUX91121',
 'OSUX88159',
 'OSUX92107',
 'OSUX91143',
 'OSUX89128',
 'OSUX92061',
 'OSUX83061',
 'OSUX87190',
 'OSUX92013',
 'OSUX87130']

In [7]:
# %%time
# # Augment each trajectory 20 times, first augment with the new point being in the circle 10 times.
# # Then augment under the condition that the new point will be on the circle.
#
# training = Augmentation.augment_trajectories_with_randomly_generated_points(training,
#                                                                            random=myRandom,
#                                                                            ids_to_augment=selected,
#                                                                            circle='in')
#
# for i in range(1, 6):
#     training = Augmentation.augment_trajectories_with_randomly_generated_points(training,
#                                                                                random=myRandom,
#                                                                                ids_to_augment=selected,
#                                                                                circle='in')
#
#
# for i in range(0, 6):
#     training = Augmentation.augment_trajectories_with_randomly_generated_points(training,
#                                                                                random=myRandom,
#                                                                                ids_to_augment=selected,
#                                                                                circle='on')
#
# training

In [8]:
# # Generate kinematic statistics on the dataset and then convert
# # the dataset from point-form representation into traj-form representation.
# pivoted_train = Statistics.pivot_stats_df(
#     dataframe=Statistics.generate_kinematic_stats(training, 'Species'),
#     target_col_name='Species')
#
# # Pivoting the test data
# pivoted_test = Statistics.pivot_stats_df(
#     dataframe=Statistics.generate_kinematic_stats(testing, 'Species'),
#     target_col_name='Species')
# pivoted_train